<a href="https://colab.research.google.com/github/Martoto/BigData_ForecastHackathon/blob/main/silicionauta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [40]:
!cp /content/drive/MyDrive/BDForecastHackathon/hackathon_2025_templates.zip /content/

In [41]:
!unzip /content/hackathon_2025_templates.zip -d /content/

Archive:  /content/hackathon_2025_templates.zip
  inflating: /content/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet  
  inflating: /content/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet  
  inflating: /content/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet  


In [3]:
import pandas as pd
file_path = '/content/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet'
df = pd.read_parquet(file_path)
display(df.head())

,pdv,premise,categoria_pdv,zipcode
0,2204965430669363375,On Premise,Mexican Rest,30741
1,5211957289528622910,On Premise,Hotel/Motel,80011
2,9024493554530757353,Off Premise,Convenience,80751
3,8659197371382902429,On Premise,Restaurant,80439
4,1400854873763881130,On Premise,Restaurant,30093


In [2]:
import pyarrow.parquet as pq
import pandas as pd

file_path = '/content/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet'

# Open the parquet file
parquet_file = pq.ParquetFile(file_path)

# Iterate over row groups (batches)
for i in range(parquet_file.num_row_groups):
    # Read a single row group into a pandas DataFrame
    df_batch = parquet_file.read_row_group(i).to_pandas()

    # Process the batch (e.g., display the head of the first batch)
    if i == 0:
        print(f"Processing batch {i}:")
        display(df_batch.head())
    # You can add your processing logic for each batch here
    # For example:
    # processed_data = process_my_batch(df_batch)
    # append_to_results(processed_data)

print("\nFinished processing batches.")

FileNotFoundError: [Errno 2] Failed to open local file '/content/part-00000-tid-6364321654468257203-dc13a5d6-36ae-48c6-a018-37d8cfe34cf6-263-1-c000.snappy.parquet'. Detail: [errno 2] No such file or directory

In [4]:
file_path = '/content/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet'
df_third = pd.read_parquet(file_path)
display(df_third.head())

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,taxes
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,0.234375
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,0.810000
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,0.405000
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,0.000000
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,2.279758
